In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy
import emission.core.get_database as edb
import logging
import folium
import math

In [ ]:
import emission.storage.timeseries.abstract_timeseries as esta

In [ ]:
# copied from mode_purpose_share.ipynb
# We select participants from real data to be users at this time
participant_uuid_obj = list(edb.get_profile_db().find({"install_group": "participant"}, {"user_id": 1, "_id": 0}))
all_users = [u["user_id"] for u in participant_uuid_obj]

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
logging.debug('There are %s users, we will plot the graphs for one of them' % len(all_users))

user = all_users[1]

In [ ]:
import emission.analysis.modelling.tour_model.cluster_pipeline as eamtc

In [ ]:
# we should experiment with different values here
radius = 300

In [ ]:
import emission.analysis.modelling.tour_model.similarity as similarity

In [ ]:
import emission.analysis.modelling.tour_model.cluster_pipeline as pipeline

In [ ]:
import emission.analysis.modelling.tour_model.featurization as featurization

In [ ]:
trips = pipeline.read_data(uuid=user)

In [ ]:
bin_trips, bins = pipeline.remove_noise(trips, radius)

In [ ]:
# bin_trips

In [ ]:
# bins

In [ ]:
# Plot all the bin trips on the map, later we should use different color for different bins, but not sure whether it works.
m = folium.Map(location=[bin_trips[0].data.start_loc["coordinates"][1], bin_trips[0].data.start_loc["coordinates"][0]],zoom_start=12, max_zoom= 30, control_scale=True)
t_index = 0
for bin in bins:    
    for i in range (len(bin)):
        layer = folium.PolyLine([[bin_trips[t_index].data.start_loc["coordinates"][1],bin_trips[t_index].data.start_loc["coordinates"][0]],
                                [bin_trips[t_index].data.end_loc["coordinates"][1],bin_trips[t_index].data.end_loc["coordinates"][0]]],weight=2, color='orange')
        t_index += 1
        layer.add_to(m)
        
m

In [ ]:
########clusters#####

In [ ]:
feat = featurization.featurization(bin_trips)
feat

In [ ]:
# cluster the data using k-means
# def cluster(data, bins)
# return feat.clusters, feat.labels, feat.data
clusters,labels,feat_data = pipeline.cluster(bin_trips,len(bins))

In [ ]:
# Copy from featurization.py, should later change it to use folium
def map_clusters(self):
#         import matplotlib.pyplot as plt
#         from matplotlib import colors as matcol
#         colormap = plt.cm.get_cmap()

    cluster_map = folium.Map(location=[self.points[0][1], self.points[0][0]],zoom_start=12, max_zoom= 30, control_scale=True)


    if self.labels:             
        for i in range(len(self.points)):
            start_lat = self.points[i][1]
            start_lon = self.points[i][0]
            end_lat = self.points[i][3]
            end_lon = self.points[i][2]
            path = [(start_lat, start_lon), (end_lat, end_lon)]
            layer = folium.PolyLine([[start_lat,start_lon],
                                     [end_lat,end_lon]],weight=2, color='orange')
            layer.add_to(cluster_map)
    cluster_map

In [ ]:
# clusters

In [ ]:
# labels

In [ ]:
# feat_data

In [ ]:
# feat.points

In [ ]:
# This doesn't show the map, I don't know why.
map_clusters(feat)